In [2]:
!pip install duckdb==1.2.2
import duckdb


In [3]:
# establish connection
conn = duckdb.connect('mimic.db', read_only=False)

conn.sql('SHOW TABLES;')

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ ADMISSIONS │
│ DRGCODES   │
│ D_ICDPROCS │
│ ICUSTAYS   │
│ PATIENTS   │
│ PROCS_ICD  │
└────────────┘

In [4]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS PRESCRIPTIONS AS
    SELECT * FROM read_csv_auto('PRESCRIPTIONS.csv');
""")

##Part 1 - 1


In [12]:
df_top_drugtype_by_ethnicity = conn.execute("""
    SELECT
        ETHNICITY,
        DRUG_TYPE,
        SUM(CAST(DOSE_VAL_RX AS DOUBLE)) AS TOTAL_DOSE,
        RANK() OVER (
            PARTITION BY ETHNICITY
            ORDER BY SUM(CAST(DOSE_VAL_RX AS DOUBLE)) DESC
        ) AS RANK
    FROM PRESCRIPTIONS
    JOIN ADMISSIONS
        ON PRESCRIPTIONS.SUBJECT_ID = ADMISSIONS.SUBJECT_ID
    WHERE DOSE_VAL_RX ~ '^[0-9.]+$'
    GROUP BY ETHNICITY, DRUG_TYPE
    HAVING TOTAL_DOSE IS NOT NULL
    QUALIFY RANK = 1
    ORDER BY ETHNICITY;
""").fetchdf()

df_top_drugtype_by_ethnicity


,ethnicity,drug_type,TOTAL_DOSE,RANK
0,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,MAIN,93134.900,1
1,ASIAN,MAIN,47430.850,1
2,BLACK/AFRICAN AMERICAN,MAIN,300000.525,1
3,HISPANIC OR LATINO,BASE,45557.000,1
4,HISPANIC/LATINO - PUERTO RICAN,MAIN,2027391.375,1
5,OTHER,MAIN,16745.000,1
6,UNABLE TO OBTAIN,BASE,26003.000,1
7,UNKNOWN/NOT SPECIFIED,MAIN,175896.500,1
8,WHITE,MAIN,1628599.290,1


## Part 1 - 2


In [8]:
df_procs_19 = conn.execute("""
    SELECT D_ICDPROCS.LONG_TITLE AS PROCEDURE_NAME, COUNT(*) AS COUNT
    FROM PROCS_ICD
    JOIN ADMISSIONS ON PROCS_ICD.HADM_ID = ADMISSIONS.HADM_ID
    JOIN PATIENTS ON PROCS_ICD.SUBJECT_ID = PATIENTS.SUBJECT_ID
    JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE FLOOR(DATE_DIFF('year', CAST(PATIENTS.DOB AS TIMESTAMP), CAST(ADMISSIONS.ADMITTIME AS TIMESTAMP))) <= 19
    GROUP BY PROCEDURE_NAME
    ORDER BY COUNT DESC
    LIMIT 3;
""").fetchdf()

df_procs_19


,PROCEDURE_NAME,COUNT
0,"Venous catheterization, not elsewhere classified",2
1,Incision of lung,1
2,Closed [endoscopic] biopsy of bronchus,1


In [9]:
df_procs_20_49 = conn.execute("""
    SELECT D_ICDPROCS.LONG_TITLE AS PROCEDURE_NAME, COUNT(*) AS COUNT
    FROM PROCS_ICD
    JOIN ADMISSIONS ON PROCS_ICD.HADM_ID = ADMISSIONS.HADM_ID
    JOIN PATIENTS ON PROCS_ICD.SUBJECT_ID = PATIENTS.SUBJECT_ID
    JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE FLOOR(DATE_DIFF('year', CAST(PATIENTS.DOB AS TIMESTAMP), CAST(ADMISSIONS.ADMITTIME AS TIMESTAMP))) BETWEEN 20 AND 49
    GROUP BY PROCEDURE_NAME
    ORDER BY COUNT DESC
    LIMIT 3;
""").fetchdf()

df_procs_20_49


,PROCEDURE_NAME,COUNT
0,"Venous catheterization, not elsewhere classified",9
1,Enteral infusion of concentrated nutritional s...,7
2,Percutaneous abdominal drainage,6


In [10]:
df_procs_50_79 = conn.execute("""
    SELECT D_ICDPROCS.LONG_TITLE AS PROCEDURE_NAME, COUNT(*) AS COUNT
    FROM PROCS_ICD
    JOIN ADMISSIONS ON PROCS_ICD.HADM_ID = ADMISSIONS.HADM_ID
    JOIN PATIENTS ON PROCS_ICD.SUBJECT_ID = PATIENTS.SUBJECT_ID
    JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE FLOOR(DATE_DIFF('year', CAST(PATIENTS.DOB AS TIMESTAMP), CAST(ADMISSIONS.ADMITTIME AS TIMESTAMP))) BETWEEN 50 AND 79
    GROUP BY PROCEDURE_NAME
    ORDER BY COUNT DESC
    LIMIT 3;
""").fetchdf()

df_procs_50_79


,PROCEDURE_NAME,COUNT
0,"Venous catheterization, not elsewhere classified",25
1,Enteral infusion of concentrated nutritional s...,22
2,Transfusion of packed cells,13


In [11]:
df_procs_80plus = conn.execute("""
    SELECT D_ICDPROCS.LONG_TITLE AS PROCEDURE_NAME, COUNT(*) AS COUNT
    FROM PROCS_ICD
    JOIN ADMISSIONS ON PROCS_ICD.HADM_ID = ADMISSIONS.HADM_ID
    JOIN PATIENTS ON PROCS_ICD.SUBJECT_ID = PATIENTS.SUBJECT_ID
    JOIN D_ICDPROCS ON PROCS_ICD.ICD9_CODE = D_ICDPROCS.ICD9_CODE
    WHERE FLOOR(DATE_DIFF('year', CAST(PATIENTS.DOB AS TIMESTAMP), CAST(ADMISSIONS.ADMITTIME AS TIMESTAMP))) >= 80
    GROUP BY PROCEDURE_NAME
    ORDER BY COUNT DESC
    LIMIT 3;
""").fetchdf()

df_procs_80plus


,PROCEDURE_NAME,COUNT
0,"Venous catheterization, not elsewhere classified",20
1,Transfusion of packed cells,13
2,Insertion of endotracheal tube,8


## Part 1 - 3


In [5]:
df_icu_overall = conn.execute("""
    SELECT
        ROUND(AVG(
            DATE_DIFF('minute',
                CAST(ICUSTAYS.INTIME AS TIMESTAMP),
                CAST(ICUSTAYS.OUTTIME AS TIMESTAMP)
            ) / 60.0
        ), 2) AS AVG_ICU_HOURS
    FROM ICUSTAYS
    WHERE INTIME IS NOT NULL AND OUTTIME IS NOT NULL;
""").fetchdf()

df_icu_overall


,AVG_ICU_HOURS
0,106.86


In [6]:
df_icu_by_gender = conn.execute("""
    SELECT
        PATIENTS.GENDER,
        ROUND(AVG(
            DATE_DIFF('minute',
                CAST(ICUSTAYS.INTIME AS TIMESTAMP),
                CAST(ICUSTAYS.OUTTIME AS TIMESTAMP)
            ) / 60.0
        ), 2) AS AVG_ICU_HOURS
    FROM ICUSTAYS
    JOIN PATIENTS ON ICUSTAYS.SUBJECT_ID = PATIENTS.SUBJECT_ID
    WHERE INTIME IS NOT NULL AND OUTTIME IS NOT NULL
    GROUP BY PATIENTS.GENDER
    ORDER BY PATIENTS.GENDER;
""").fetchdf()

df_icu_by_gender

,gender,AVG_ICU_HOURS
0,F,132.96
1,M,84.33


In [7]:
df_icu_by_ethnicity = conn.execute("""
    SELECT
        ADMISSIONS.ETHNICITY,
        ROUND(AVG(
            DATE_DIFF('minute',
                CAST(ICUSTAYS.INTIME AS TIMESTAMP),
                CAST(ICUSTAYS.OUTTIME AS TIMESTAMP)
            ) / 60.0
        ), 2) AS AVG_ICU_HOURS
    FROM ICUSTAYS
    JOIN ADMISSIONS ON ICUSTAYS.HADM_ID = ADMISSIONS.HADM_ID
    WHERE INTIME IS NOT NULL AND OUTTIME IS NOT NULL
    GROUP BY ADMISSIONS.ETHNICITY
    ORDER BY AVG_ICU_HOURS DESC;
""").fetchdf()

df_icu_by_ethnicity

,ethnicity,AVG_ICU_HOURS
0,UNABLE TO OBTAIN,320.57
1,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,272.10
2,BLACK/AFRICAN AMERICAN,184.24
3,HISPANIC OR LATINO,179.03
4,UNKNOWN/NOT SPECIFIED,118.21
5,WHITE,99.13
6,ASIAN,93.36
7,HISPANIC/LATINO - PUERTO RICAN,77.83
8,OTHER,22.22
